In [1]:
from __future__ import division
from more_itertools import peekable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy as scp
import pickle
import magenta
import os, time, re, json
%matplotlib inline
from IPython.core.display import display, HTML
### change width of notebook display
# display(HTML("<style>.container { width:70% !important; }</style>"))

import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# for exposing API
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from flask import Flask
from flask_restful import Resource, Api

JUPYTER_PICKLE_FILE = "config/shared_jupyter_data.pkl"
def write_shared_jupyter(key, value, path=JUPYTER_PICKLE_FILE, overwrite=False):
    if (os.path.exists(path)):
        with open(path, "rb") as fp:
            shared_jupyter_data = pickle.load(fp)
        if overwrite:
            shared_jupyter_data = {key: value}
        else:
            shared_jupyter_data[key] = value
    else:
        shared_jupyter_data = {key: value}
    with open(path, 'wb') as fp: 
        pickle.dump(shared_jupyter_data, fp)

def read_shared_jupyter(key=None, path=JUPYTER_PICKLE_FILE):
    if (os.path.exists(path)):
        with open(path, "rb") as fp:
            shared_jupyter_data = pickle.load(fp)
            if key is not None:
                if key in shared_jupyter_data:
                    return(shared_jupyter_data[key])
                else:
                    print("Not found!")
                    return(None)
            else:
                return(shared_jupyter_data)
    else:
        print("No data")

def pandasToJson(df):
    return(df.to_json(orient="split"))
def jsonToPandas(json):
    return(pd.read_json(json, orient="split"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

FIG_WIDTH = 1200
FIG_HEIGHT = 800

PITCH_MIN = 20
PITCH_MAX = 120
VELOCITY_MIN = 0
VELOCITY_MAX = 120

def hheader(x):
    print("#########################################")
    print("### {}".format(x))
    print("#########################################")

# Magenta dependencies:
# https://github.com/magenta/magenta

# Magenta uses pretty_midi to deal with midi files
import pretty_midi

C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\librosa\util\decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
C:\Users\echow\Miniconda3\envs\tensorflow\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Could

# Read in data

In [7]:
df_raw = pd.read_csv("music_stream_sample.csv")
print(df_raw.shape)
df_raw.head()

(600, 35)


,Unnamed: 0,streaming_start_sec,piece_num,start_sec,start_min,start_mean,start_median,start_max,start_str_concat,end_min,end_mean,end_median,end_max,end_str_concat,pitch_min,pitch_mean,pitch_median,pitch_max,pitch_str_concat,velocity_min,velocity_mean,velocity_median,velocity_max,velocity_str_concat,duration_min,duration_mean,duration_median,duration_max,duration_str_concat,canonical_composer_mode,split_mode,year_mode,total_duration_mode,curr_filename_mode,piece_changed
0,905,905,0,905,905.00104,905.466111,905.39792,905.96250,"905.00104,905.01146,905.125,905.13542,905.2583...",905.09479,905.589166,905.46667,906.24062,"905.13229,905.09479,905.20938,905.24688,905.35...",38.0,56.466667,62.0,74.0,"42,62,45,66,69,48,38,66,69,47,74,43,64,47,67",69.0,76.066667,73.0,88.0,"70,72,73,69,69,73,82,83,77,88,87,72,76,73,77",0.03438,0.123055,0.084380,0.57604,"0.13125,0.08333,0.08438,0.11146,0.1,0.05833,0....",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
1,906,906,0,906,906.07396,906.505530,906.50104,906.90938,"906.07396,906.09688,906.20104,906.23125,906.36...",906.13854,906.619231,906.68333,907.03854,"906.13854,906.17396,906.24167,906.37708,906.49...",40.0,59.615385,66.0,76.0,"50,71,40,67,71,48,76,45,66,48,69,52,72",68.0,80.230769,82.0,89.0,"76,82,87,82,80,88,89,79,82,68,70,78,82",0.04063,0.113702,0.077080,0.53750,"0.06458,0.07708,0.04063,0.14583,0.13646,0.0718...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
2,907,907,0,907,907.03229,907.455821,907.45104,907.87604,"907.03229,907.04896,907.175,907.175,907.29688,...",907.09479,907.578798,907.57917,907.97708,"907.09479,907.45208,907.21979,907.31042,907.33...",42.0,61.941176,67.0,79.0,"42,69,50,72,48,78,75,47,79,69,50,74,48,71,55,5...",9.0,72.000000,79.0,88.0,"86,84,79,81,79,83,9,83,86,51,60,71,59,78,70,77,88",0.02292,0.122978,0.108330,0.40313,"0.0625,0.40313,0.04479,0.13542,0.03542,0.12708...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
3,908,908,0,908,908.01042,908.444297,908.43333,908.97500,"908.01042,908.02708,908.03333,908.13958,908.14...",908.08438,908.565187,908.55417,909.10938,"908.12396,908.14792,908.08438,908.20312,908.24...",50.0,63.263158,60.0,77.0,"77,55,53,59,74,57,60,76,57,72,71,69,54,50,66,6...",61.0,74.000000,76.0,89.0,"88,63,66,68,89,61,79,86,62,61,70,76,64,76,82,7...",0.02917,0.120888,0.113540,0.28229,"0.11354,0.12083,0.05104,0.06354,0.09375,0.0291...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0
4,909,909,0,909,909.08438,909.510548,909.49844,909.96146,"909.08438,909.10104,909.18958,909.2375,909.362...",909.14271,909.694401,909.61667,910.58750,"909.14271,909.18646,909.37604,909.29479,909.5,...",38.0,59.437500,63.0,71.0,"55,71,55,69,48,64,67,50,67,62,69,38,60,66,43,67",70.0,81.687500,82.0,87.0,"85,82,70,81,77,87,84,82,83,87,80,78,82,83,81,85",0.04375,0.183855,0.120315,0.62604,"0.05833,0.08542,0.18646,0.05729,0.1375,0.06146...",Johann Sebastian Bach,train,2004.0,967.16405,2004__MIDI-Unprocessed_SMF_02_R1_2004_01-05_OR...,0


# Structural breaks method 1: Kalman filter + CUSUM

In [5]:
""" Model parameters """

from scipy.stats import multivariate_normal
from statsmodels.tsa.stattools import acovf
from scipy.ndimage.interpolation import shift
np.random.seed(42)
np.set_printoptions(suppress=True) 

def pandas_fill(arr):
    df = pd.DataFrame(arr)
    df = df.fillna(method='ffill', axis=1, inplace=False)
    out = df.to_numpy()
    return out

# for now, keep the noise matrices as constants (can improve later)

# observations x: note, different dimension from Z (unlike sample code)
df_X = None
x_obs = None
[n,d] = [None, None]
x_obs_nonna = None

# number of lags (minus one)
L = 14

### Latent state equation - vector z_t
# z_t = A_t z_{t-1} + <other regressors> + w_t
At = np.eye(L) # (L x L) - paper says to leave this as identity (random walk)
Qt = np.diag(np.ones(L)) # (L x L) covariance of errors in state eq. - TUNE THIS

### Observation equation - scalar x_t
# x_t = C_t z_t + <other regressors> + v_t
Ct = np.ones((1,L)) # (1 x L) - ARIMA lags - WILL OVERRIDE
Rt = np.diag(np.ones(1)) # (1 x 1)

# Simulate some plausible values for Qt and Rt
# (could make Rt self-updating if time)
Qt = np.diag(np.random.gamma(1,0.1,size=L)) # covar. mat is symmetric
Rt = np.random.gamma(3, 4, size=np.diag(np.ones(1)).shape)

# Initial conditions (for the state variable z, stored in mu vars)
# z0 = x_obs[1:(L+1)]
# initialize to equal weight on all lags
z0 = np.repeat(1/L, L).reshape(-1, 1) # make state variable the WEIGHTS on lags NOT the LAGS themselves
P0 = Qt

#Objects to store predictions and filtering locations
Z = None
Zpred = None
Xpred = None

# store predictions
z = z0 # predicted zhat, ()
P = P0 # covariance of zhat, dim(L, L)
Ct = Ct0 = None